# Cálculo de armónicos esféricos utilizando computación cuántica.   
## TFE: Computación cuántica aplciada al análisis espacial en electroencefalografía - Procesador cuántico Real   
## Master universitario en computación cuántica (UNIR)   
### Francisco Vidal Requejo   
### Antonio José Ruz Hervás   

En este notebook se desarrolla el modelo teórico propuesto en el trabajo para la evaluación de armónicos esféricos mediante la utilización de computación cuántica. Se utiliza aquí un procesador cuántico real para realizar el experimento. El resultado se guarda en un el archivo especificado en "strNombreArchivo" y se permite la reanudación del experimento donde se quedó.

Primeramente, es necesario instalar la librería "pennylane", que será utilizada para los circuitos cuánticos.

In [6]:
%pip install pennylane > o
%pip install pennylane-qiskit > o
%pip install qiskit_aer >o

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for qiskit_aer from https://files.pythonhosted.org/packages/10/3a/09935130dbb72ec4154a067f5611c83ae25eb1af1ff9ffb5e445b33afb33/qiskit_aer-0.12.2-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   ----- ---------------------------------- 1.3/9.6 MB 28.3 MB/s eta 0:00:01
   ----------------- ---------------------- 4.1/9.6 MB 44.3 MB/s eta 0:00:01
   ----------------- ---------------------- 4.2/9.6 MB 44.9 MB/s eta 0:00:01
   ----------------- ---------------------- 4.2/9.6 MB 44.9 MB/s eta 0:00:01
   ----------------- ---------------------- 4.2/9.6 MB 44.9 MB/s eta 0:00:01
   ----------------- ---------------------- 4.2/9.6 MB 44.9 MB/s eta 0:00:01
   ----------------- ---------------------- 4.2/9.6 MB 44.9 MB/s eta 0:00:01
   ----------------- ---------------------- 4

In [2]:
strNombreArchivo="results.csv"



Importación de librerías utilizadas en este archivo.

In [3]:
import pennylane as qml
from pennylane import numpy as np
import math
import sympy as sp
from scipy.special import lpmv
from scipy.special import sph_harm
import csv

Ángulos utilizados para el cálculo de los armónicos esféricos. La primera coordenada corresponde con la coordenada azimutal y la segunda con la polar. Estos ángulos se corresponden con la posición de 64 electrodos de un EEG utilizando el "sistema 10-10" para elegir la posición de los mismos, considerando la cabeza como una esfera.

In [4]:
angles = [#['Fc5.',-71,-21]]#,
 #['Fc3.',-50,-28],
 #['Fc1.',-32,-45],
 #['Fcz.',23,90],
 #['Fc2.',32,45],
 #['Fc4.',50,28],
 #['Fc6.',71,21],
 #['C5..',-69,0],
 #['C3..',-46,0],
 #['C1..',23,0],
 #['Cz..',0,0],
 #['C2..',23,0],
 #['C4..',46,0],
 ['C6..',69,0]]#,
 #['Cp5.',-71,21],
 #['Cp3.',-50,28],
 #['Cp1.',-32,45],
 #['Cpz.',23,-90],
 #['Cp2.',32,-45],
 #['Cp4.',50,-28],
 #['Cp6.',71,-21],
 #['Fp1.',-92,-72],
 #['Fpz.',92,90],
 #['Fp2.',92,72],
 #['Af7.',-92,-52],
 #['Af3.',-74,-67],
 #['Afz.',69,90],
 #['Af4.',74,67],
 #['Af8.',92,52],
 #['F7..',-92,-36],
 #['F5..',-75,-41],
 #['F3..',-60,-51],
 #['F1..',-50,-68],
 #['Fz..',46,90],
 #['F2..',50,68]]#,
 #['F4..',60,51],
 #['F6..',75,41],
 #['F8..',92,36],
 #['Ft7.',-92,-18],
 #['Ft8.',92,18],
 #['T7..',-92,0],
 #['T8..',92,0],
 #['T9..',-115,0],
 #['T10.',115,0],
 #['Tp7.',-92,18],
 #['Tp8.',92,-18],
 #['P7..',-92,36],
 #['P5..',-75,41],
 #['P3..',-60,51],
 #['P1..',-50,68],
 #['Pz..',46,-90],
 #['P2..',50,-68],
 #['P4..',60,-51],
 #['P6..',75,-41],
 #['P8..',92,-36],
 #['Po7.',-92,54],
 #['Po3.',-74,67],
 #['Poz.',69,-90],
 #['Po4.',74,-67],
 #['Po8.',92,-54],
 #['O1..',-92,72],
 #['Oz..',92,-90],
 #['O2..',92,-72],
 #['Iz..',115,-90]
 #]

#Versión de los ángulos en radianes.
anglesRad = []
piFactor = math.pi/180
for x in angles:
    anglesRad.append([x[0], x[1] * piFactor, x[2] * piFactor])
anglesRad

[['C6..', 1.2042771838760873, 0.0]]

In [5]:
nCubits = 4
nShots = 20000
strDevice = "qiskit.ibmq"
strBackend="ibmq_belem"

#---------- ----------------AQUÍ TU TOKEN DE IBM -----------------------------------
strToken="7a60a88cb869db32476e0939ad7d1cecfeeff8d58659acccd3d7836f0a2979e6992786fe258164ec12c3217ca0b233781f65cce5e8ee4e9046c24b7e3b028768"
#dev = qml.device ("default.qubit", wires = nCubits)
dev = qml.device (strDevice, backend=strBackend, ibmqx_token=strToken, wires = nCubits, shots = nShots)

@qml.qnode(dev)
def CircuitoCoseno(theta, nCubits = 3):
  """
  Implementar un circuito cuántico con una puerta de ratación sobre el eje Y para todos los cúbits. El mismo ángulo se aplica a todos los cúbits.
  Parámetros:
    theta: Ángulo en radianes que se quiere aplicar en una rotación sobre el eje Y.
    nCubits: Cantidad de cúbits del circuito

  Retorno:
    Array de probabilidades de cada uno de los estados.
  """
  for i in range(nCubits):
    qml.RY(theta, wires = i)
  return qml.probs(wires=range(nCubits))

def CircuitoSeno(theta, nCubits=3):
  """
  Implementar un circuito cuántico con una puerta de rotación de pi/2 - theta sobre el eje Y para todos los cúbits. El mismo ángulo se aplica a todos los cúbits.
  Parámetros:
    theta: Ángulo en radianes que se quiere aplicar como una rotación pi/2 - theta sobre el eje Y.
    nCubits: Cantidad de cúbits del circuito

  Retorno:
    Array de probabilidades de cada uno de los estados.
  """
  return CircuitoCoseno((math.pi/2) - theta, nCubits)

def ExpPhi(phi, m):
  """
  Obtener el valor en coordenadas cartesianas de un número complejo en forma polar de la manera e^{im} a través de dos circuitos cuánticos de un único cúbit utilizando la igualdad e^{im} = cos(im) + i·sen(im) .

  Parámetros:
    phi: el ángulo base
    m: valor a multiplicar al ángulo.

  Retorno:
    Una tupla con el valor real e imagino del número complejo en coordenadas cartesianas.
  """
  C = CircuitoCoseno(phi * m, 1)
  S = CircuitoSeno(phi * m, 1)
  return (C[0] - C[1], S[0] - S[1])


def EvalAsocLegendre4(theta):
  """
  Evaluar los Polinomios asociados de legendre en un ángulo dado utilizando un circutio de 4 cúbits. Esto generará todos los polinomios asociados de Legendre hasta l = 4.

  Parámetros: 
    theta: ángulo en radiantes sobre el cual calcular los polinomios asociados de Legendre.

  Retorno:
    Un diccionario en el que cada elemento tiene una etiqueta de la forma "P_l_m" indicando el polinomio asociado de Legendre al que hace referencia y cuyo valor es el resultado de la evaluación de dicho polinómio sobre el ángulo dado.
  """
  nCubits = 4
  C = CircuitoCoseno(abs(theta), nCubits) #importante que el ángulo sea siempre positivo.
  S = CircuitoSeno(abs(theta), nCubits)
  PolinomiosLegendre = {'P_0_0': 1}
  PolinomiosLegendre['P_1_0']= C[0] + 2*C[1] -2*C[7] -C[15]
  PolinomiosLegendre['P_1_1']= -S[0] - 2*S[1] + 2*S[7] + S[15]
  PolinomiosLegendre['P_2_0']= (3*(C[0] - 2*C[3] + C[15]) - 1)/2
  PolinomiosLegendre['P_2_1']= 3*(C[0] + 2*C[1] - 2*C[7] - C[15])*(-S[0] -2*S[1] + 2*S[7] + S[15])
  PolinomiosLegendre['P_2_2']= 3*(S[0] - 2*S[3] + S[15])
  PolinomiosLegendre['P_3_0']= C[0] - 8*C[1] + 8*C[7] - C[15]
  PolinomiosLegendre['P_3_1']= -(3/2)*(-S[0] + 18*S[1] -18*S[7] + S[15])
  PolinomiosLegendre['P_3_2']= 15*(4*C[1] - 4*C[7])
  PolinomiosLegendre['P_3_3']= -15*(S[0] -2*S[1] + 2*S[7] - S[15])
  PolinomiosLegendre['P_4_0']= (1/8)*(5*C[0] - 140*C[1] + 270*C[3] -140*C[7] + 5*C[15] + 3)
  PolinomiosLegendre['P_4_1']= -(5/2)*(7*(C[0] +2*C[1] -2*C[7] - C[15])*(4*S[1]-4*S[7]) + 3*(C[0]+2*C[1]-2*C[7]-C[15])*(-S[0]-2*S[1]+2*S[7]+S[15]))
  PolinomiosLegendre['P_4_2'] = (15/2)*(56*C[1] - 112*C[3] + 56*C[7] - S[0] - 28*S[1] + 58*S[3] - 28*S[7] - S[15])
  PolinomiosLegendre['P_4_3'] = -105*(C[0] + 2*C[1] - 2*C[7] - C[15])*(S[0] - 2*S[1] + 2*S[7] - S[15])
  PolinomiosLegendre['P_4_4'] = 105*(S[0] - 4*S[1] + 6*S[3] - 4*S[7] + S[15])

  #Polinomios de "m" negativo
  for l in range(1, nCubits + 1):
    for m in range(1, l+1):
      PolinomiosLegendre['P_' + str(l) + '_-' + str(m)] = ((-1)**(m) * (math.factorial(l-m)/math.factorial(l+m))) * PolinomiosLegendre['P_' + str(l) + '_' + str(m)]
  return PolinomiosLegendre

def EvalEsfericos4(theta, phi):
  """
  Realiza la evaluación de armónicos esféricos mediante cirucitos cuánticos de 4 cubits. Se obtiene el resultado de evalluar los armónicos esféricos para los ángulos dados hasta l = 4.

  Parámetros:
    theta: ángulo én radianes de la coordenada polar.
    phi: ángulo en radianes de la coordenada azimutal.

  Retorno:
    Un diccionario en el que cada elemento tiene una etiqueta de la forma "Y_l_m" indicando el armónico esférico al que hace referencia y cuyo valor es el resultado de la evaluación de dicho armónico esféricos sobre los ángulos dados.
  """
  l = 4
  legendre = EvalAsocLegendre4(theta)
  AE = {}
  e = []
  for m in range(l+1):
    if (m==0):
      e.append((1, 0))
    else:
      e.append(ExpPhi(phi, (m)))
  for k in range(l+1):
    for m in range(0, k+1):
      coef = math.sqrt(((2*k + 1)*math.factorial(k - (m))) / (4*math.pi * math.factorial(k+(m)))) #Coeficiente de normalización
      legendre_normalizado = (coef *  legendre['P_' + str(k) + '_' + str(m)])
      AE['Y_' + str(k) + '_' + str(m)] = ( legendre_normalizado * e[m][0], legendre_normalizado * e[m][1])
      #lo mismo para "m" negativos
      coef = math.sqrt(((2*k + 1)*math.factorial(k + (m))) / (4*math.pi * math.factorial(k-(m))))
      legendre_normalizado = (coef *  legendre['P_' + str(k) + '_' + str(-m)])
      AE['Y_' + str(k) + '_' + str(-m)] = ( legendre_normalizado * e[m][0], legendre_normalizado * (-e[m][1]))

  return AE

ModuleNotFoundError: No module named 'qiskit_aer'

Se calculan los armónicos esféricos utilizando circuitos cuánticos según la propuesta del trabajo. Para cada electrodo se guardan en un archivo los armónicos esféricos calculados. Si se interrumpe el proceso, se recupera la última ejecución según los datos del archivo.

In [ ]:
anglesTmp = {}
for x in anglesRad:
    anglesTmp[x[0]] = [float(x[1]), float(x[2])]
resultadoAE = {}

for x in anglesTmp:
    xVal = anglesTmp[x]
    AE = EvalEsfericos4(xVal[1], xVal[0])
    calculadosAE = []
    for armonicos in AE.values():
        calculadosAE.append(complex(armonicos[0], armonicos[1]))
    resultadoAE[x] = calculadosAE
    
resultadoAE


#Antonio, lo que salga aquí copiamelo a un txt (1/3)

{'F2..': [(0.28209479177387814+0j),
  (0.18303372251841052+0j),
  (-0.2059083956331595-0.2453920702411278j),
  (0.2059083956331595-0.2453920702411278j),
  (-0.18261477573960364+0j),
  (-0.1724783043718558-0.2055516388797168j),
  (0.17247830437185582-0.20555163887971686j),
  (-0.0576630559705845+0.3270234409895744j),
  (-0.0576630559705845-0.3270234409895744j),
  (-0.321296193400511+0j),
  (0.057464998121026976+0.06848411795915345j),
  (-0.057464998121026976+0.06848411795915345j),
  (-0.05715077078835502+0.32411812734913675j),
  (-0.057150770788355-0.3241181273491367j),
  (0.2880031991544722-0.1662787245593079j),
  (-0.2880031991544722-0.1662787245593079j),
  (-0.05507874744953035+0j),
  (0.2131102796672191+0.25397494141216526j),
  (-0.2131102796672191+0.25397494141216526j),
  (0.0008833625086544224-0.005009797735463719j),
  (0.0008833625086544226+0.00500979773546372j),
  (0.3236636919844241-0.18686731969411532j),
  (-0.323663691984424-0.1868673196941153j),
  (-0.30732268467645774-0.111

Se calculan los armónicos esféricos mediante la función correspondiente de la librería "scipy" de python. Hay que tener en cuenta que para esta función "theta" es la coordenada azimutal y "phi" la polar, al revés de lo considerado anteriormente.

In [ ]:
nCubits = 4
m = []
v = []
for l in range(nCubits + 1):
    for n in range(0, l+1):
        m.append(n)
        if (n>0):
            m.append(-n)
            v.append(l)
        v.append(l)
resultadoSph = {}
for x in anglesRad:
    resultadoSph[x[0]]= sph_harm(m, v, x[1], x[2])
resultadoSph
#Antonio, lo que salga aquí copiamelo a un txt (2/3)

{'F2..': array([ 0.28209479+0.j        ,  0.18303372+0.j        ,
        -0.2059084 -0.24539207j,  0.2059084 -0.24539207j,
        -0.18261478+0.j        , -0.1724783 -0.20555164j,
         0.1724783 -0.20555164j, -0.05766306+0.32702344j,
        -0.05766306-0.32702344j, -0.32129619+0.j        ,
         0.057465  +0.06848412j, -0.057465  +0.06848412j,
        -0.05715077+0.32411813j, -0.05715077-0.32411813j,
         0.2880032 -0.16627872j, -0.2880032 -0.16627872j,
        -0.05507875+0.j        ,  0.21311028+0.25397494j,
        -0.21311028+0.25397494j,  0.00088336-0.0050098j ,
         0.00088336+0.0050098j ,  0.32366369-0.18686732j,
        -0.32366369-0.18686732j, -0.30732268-0.11185631j,
        -0.30732268+0.11185631j])}

In [ ]:
error = {}
for x in resultadoAE:
    error[x] = (resultadoAE[x] - resultadoSph[x])
error
#Antonio, lo que salga aquí copiamelo a un txt (3/3)

{'F2..': array([ 0.00000000e+00+0.00000000e+00j,  1.11022302e-16+0.00000000e+00j,
         0.00000000e+00-8.32667268e-17j,  0.00000000e+00-8.32667268e-17j,
         1.11022302e-16+0.00000000e+00j, -8.32667268e-17-1.66533454e-16j,
         1.38777878e-16-2.49800181e-16j,  0.00000000e+00+5.55111512e-17j,
        -1.38777878e-17-1.11022302e-16j, -4.44089210e-16+0.00000000e+00j,
         6.24500451e-17+9.71445147e-17j, -6.93889390e-17+1.11022302e-16j,
        -6.24500451e-17+3.33066907e-16j, -3.46944695e-17-2.22044605e-16j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        -1.17961196e-16+0.00000000e+00j,  1.66533454e-16+3.33066907e-16j,
        -1.66533454e-16+3.33066907e-16j,  1.32272665e-17-7.63278329e-17j,
         1.34441069e-17+7.71951947e-17j,  1.11022302e-16-8.32667268e-17j,
        -5.55111512e-17-5.55111512e-17j,  0.00000000e+00-4.16333634e-17j,
         0.00000000e+00+2.77555756e-17j])}

Se comparan los resultados para obtener el error. Para simplificar se obtendrá el máximo error encontrado.

In [ ]:
error = {}
for x in resultadoAE:
    error[x] = max(resultadoAE[x] - resultadoSph[x])
error

{'F2..': (1.6653345369377348e-16+3.3306690738754696e-16j)}

Como se puede observar el máximo error es del orden de $10^{-16}$. Dicho error resulta despreciable y viene dado por la limitación en la precisión de los cálculos realizados mediante la librería "scipy".

In [ ]:
tolerancia = 10**(-15)
print("Electrodos con error superior a " + str(tolerancia) + ":")
print("-----------------------------------------")
noError = True
for x in error:
    if abs(error[x]) > tolerancia:
        print(x + "-->" + str(error[x]))
        noError = False
if noError:
    print("No hay errores superiores a " + str(tolerancia) + ".")
print("-----------------------------------------")

Electrodos con error superior a 1e-15:
-----------------------------------------
No hay errores superiores a 1e-15.
-----------------------------------------
